# Artificial Intelligence - Fall 2020 - Laboratory 09

## _First Order Predicate Logic:   Forward chaining in Rule-Based Systems_

c: Alexandra Dobrescu <alexandramaria.digital@gmail.com>

## Introduction

The purpose of this laboratory is to understand and implement theorem proving using forward chaining.

###  Definite clauses. Rules. Facts.

In this laboratory we will use a certain type of formulas, more precisely **definite clauses**. These are Horn clauses with exactly one positive literal.

$$\neg p_1 \vee \neg p_2 \vee \ldots \vee \neg p_N \vee c$$

This leads us to:

$$\left( p_1 \wedge p_2 \wedge \ldots \wedge p_N \right) \rightarrow c$$

The particular rule-like form of clauses is similar to the **rules**.

We will call **rule** a clause defined with at least one negative literal and we will call **fact** a clause defined with only one literal (the positive one).

### Problem to be solved

_Given a knowledge base_ `kb` _consisting of defined clauses (facts and rules), prove the theorem_ `t`.

## Useful functions from previous laboratories

### Task 0

Save the solution of `lab 8` (*Representation and Unification*) with the name `Lab08.py`. We will also use the functions already implemented in `Homework3_Resolution`. The important functions for today are:
  - `make_var`,` make_const`, `make_atom` - useful for building atoms. 
  For example, $$ Weather (tomorrow, Cold) $$ is constructed as follows:
  
`make_atom (" Weather ", make_var (" tomorrow "), make_const (" Cold "))`
 
  - `unify` and` substitute` - useful for calculating the most general unifier for two formulas and for applying a substitution
  - `is_positive_literal` and` is_negative_literal`
  - `add_statement` - adds a defined clause to a knowledge base. We have two variants of use in this laboratory:
    * `add_statement (kb, Atom)` - adds the fact to Atom in kb
    * `add_statement (kb, C, P1, P2, Pn)` - add in kb the rule $\left(p_1 \wedge p_2 \wedge p_N \right) \rightarrow c$ as:
   
    `make_or (make_neg (P1), make_neg (P2), make_neg (Pn), C)`

### Task 1 (optional)

Make the following change to the `unify` function: the function header

```python
def unify(f1, f2):
    ...
    subst = {}
```

should be changed to receive a third parameter, a starting substitution.

```python
def unify(f1, f2, subst=None):
    if not subst:
        subst = {}
```
**Remember:** After modifying `unify` to download `Lab08` again as a Python file and restart Kernel in this file.

In [29]:
from Lab08 import make_var, make_const, make_atom, make_or, make_neg, \
                is_variable, is_constant, is_atom, is_function_call, \
                print_formula, get_args, get_head, get_name, get_value ,\
                unify, substitute
from Homework3_Resolution import add_statement, is_positive_literal, is_negative_literal, \
                make_unique_var_names, print_KB
from LPTester import *

## Knowledge base

Complete the representation of the knowledge base below knowing that it corresponds to the following statements:

[TODO 2.1:] *If it rained two days in a row, the third day will be sunny.* 

[TODO 2.2:] *If it was sunny three days in a row, it will rain on the fourth day.* 

[TODO 2.3:] *A student always goes to the mountains if it is sunny on a weekend day. Those who go to the mountains and practise a winter sport will have activities related to that winter sport.*

*Mary and Kevin are students. Mary plays volleyball and skiing, and Kevin skis and sledging. Volleyball is a summer sport, while skiing and sledging are winter sports. It's raining on Friday; Monday, Tuesday and Wednesday is sunny.*

### Task 2:

Fill in the clauses defined for the first 3 sentences below.

In [30]:
def get_sports_kb():
    sports_kb = []
    # Predicate 'Consecutive'
    add_statement(sports_kb, make_atom('Consecutive', make_const('Monday'), make_const('Tuesday')))
    add_statement(sports_kb, make_atom('Consecutive', make_const('Tuesday'), make_const('Wednesday')))
    add_statement(sports_kb, make_atom('Consecutive', make_const('Wednesday'), make_const('Thursday')))
    add_statement(sports_kb, make_atom('Consecutive', make_const('Thursday'), make_const('Friday')))
    add_statement(sports_kb, make_atom('Consecutive', make_const('Friday'), make_const('Saturday')))
    add_statement(sports_kb, make_atom('Consecutive', make_const('Saturday'), make_const('Sunday')))
    # Predicate 'Weekend'
    add_statement(sports_kb, make_atom('Weekend', make_const('Saturday')))
    add_statement(sports_kb, make_atom('Weekend', make_const('Sunday')))
    # Predicate 'Rain'
    add_statement(sports_kb, make_atom('Rain', make_const('Friday')))
    # TODO 2.1: If it rained two days in a row, the third day will be sunny.
    add_statement(sports_kb, make_or(make_neg(make_atom('Rain', make_var('x'))),\
                                     make_neg(make_atom('Rain', make_var('y'))),\
                                     make_neg(make_atom('Consecutive', make_var('x'), make_var('y'))),\
                                     make_neg(make_atom('Consecutive', make_var('y'), make_var('z'))),\
                                     make_atom('Sunny', make_var('z'))))
    # Predicate 'Sunny'
    add_statement(sports_kb, make_atom('Sunny', make_const('Monday')))
    add_statement(sports_kb, make_atom('Sunny', make_const('Tuesday')))
    add_statement(sports_kb, make_atom('Sunny', make_const('Wednesday')))
    # TODO 2.2: If it was sunny three days in a row, it will rain on the fourth day.
    add_statement(sports_kb, make_or(make_neg(make_atom('Sunny', make_var('x'))),\
                                     make_neg(make_atom('Sunny', make_var('y'))),\
                                     make_neg(make_atom('Sunny', make_var('z'))),\
                                     make_neg(make_atom('Consecutive', make_var('x'), make_var('y'))),\
                                     make_neg(make_atom('Consecutive', make_var('y'), make_var('z'))),\
                                     make_neg(make_atom('Consecutive', make_var('z'), make_var('n'))),\
                                     make_atom('Rain', make_var('n'))))
    # Predicate 'Student'
    add_statement(sports_kb, make_atom('Student', make_const('Mary')))
    add_statement(sports_kb, make_atom('Student', make_const('Kevin')))
    # GoesToMountain (who, when)
    # TODO 2.3: A student always goes to the mountains if it is sunny on a weekend day.
    add_statement(sports_kb, make_or(make_neg(make_atom('Student', make_var('who'))),\
                                     make_neg(make_atom('Sunny', make_var('day'))),\
                                     make_neg(make_atom('Weekend', make_var('day'))),\
                                     make_atom('GoesToMountain', make_var('who'), make_var('day'))))
    # Predicate 'SummerSport'
    add_statement(sports_kb, make_atom('SummerSport', make_const('Volleyball')))
    # Predicate 'WinterSport'
    add_statement(sports_kb, make_atom('WinterSport', make_const('Skiing')))
    add_statement(sports_kb, make_atom('WinterSport', make_const('Sledgging')))
    # Predicate 'PractiseSport'
    add_statement(sports_kb, make_atom('PractiseSport', make_const('Kevin'), make_const('Skiing')))
    add_statement(sports_kb, make_atom('PractiseSport', make_const('Kevin'), make_const('Sledgging')))
    add_statement(sports_kb, make_atom('PractiseSport', make_const('Mary'), make_const('Skiing')))
    add_statement(sports_kb, make_atom('PractiseSport', make_const('Mary'), make_const('Volleyball')))
    # Predicate 'Activity'
    add_statement(sports_kb, make_atom('Activity', make_var('who'), make_var('what'), make_var('when')),
                  make_atom('GoesToMountain', make_var('who'), make_var('when')),
                  make_atom('PractiseSport', make_var('who'), make_var('what'))
                 )
    make_unique_var_names(sports_kb)
    return sports_kb


print("This is how the knowledge base is presented:")
skb = get_sports_kb()
print_KB(skb)
print("==================== \n Inside the KB we can notice:")
print("" + "".join([(str(s) + "\n") for s in skb]))

This is how the knowledge base is presented:
OK: Added statement Consecutive(Monday, Tuesday)
OK: Added statement Consecutive(Tuesday, Wednesday)
OK: Added statement Consecutive(Wednesday, Thursday)
OK: Added statement Consecutive(Thursday, Friday)
OK: Added statement Consecutive(Friday, Saturday)
OK: Added statement Consecutive(Saturday, Sunday)
OK: Added statement Weekend(Saturday)
OK: Added statement Weekend(Sunday)
OK: Added statement Rain(Friday)
OK: Added statement (V ~Rain(?x) ~Rain(?y) ~Consecutive(?x, ?y) ~Consecutive(?y, ?z) Sunny(?z))
OK: Added statement Sunny(Monday)
OK: Added statement Sunny(Tuesday)
OK: Added statement Sunny(Wednesday)
OK: Added statement (V ~Sunny(?x) ~Sunny(?y) ~Sunny(?z) ~Consecutive(?x, ?y) ~Consecutive(?y, ?z) ~Consecutive(?z, ?n) Rain(?n))
OK: Added statement Student(Mary)
OK: Added statement Student(Kevin)
OK: Added statement (V ~Student(?who) ~Sunny(?day) ~Weekend(?day) GoesToMountain(?who, ?day))
OK: Added statement SummerSport(Volleyball)
OK: Ad

## Auxiliary functions

### Task 3

Implement the functions `get_premises`,` get_conclusion`, `is_fact` and` is_rule`. All of these receive a `definite clause` 
(in the given knowledge base, it can be a single atom or a disjunction of literals) and return what their name specifies.

In [31]:
def get_premises(formula):
    # TODO
    premises = []
    for arg in get_args(formula):
        if is_negative_literal(arg):
            premises.append(get_args(arg)[0])
    return premises

def get_conclusion(formula):
    # TODO
    for arg in get_args(formula):
        if is_positive_literal(arg):
            return arg
    return None

def is_fact(formula):
    # TODO
    if len(get_args(formula)) == 1 and not is_negative_literal(get_args(formula)[0]):
        return True
    return False

def is_rule(formula):
    # TODO
    for arg in get_args(formula):
        if is_negative_literal(arg):
            return True
    return False

# Test!
# formula: P(x) ^ Q(x) -> R(x)
f = make_or(make_neg(make_atom("P", make_var("x"))), make_neg(make_atom("Q", make_var("x"))), make_atom("R", make_var("x")))
print("".join([(print_formula(p, True) + " ; ") for p in get_premises(f)])[:-3]) # Should be P(?x) ; Q(?x)
print_formula(get_conclusion(f)) # Should be R(?x)
print(is_rule(f)) # must be True
print(is_fact(get_conclusion(f))) # must be True

P(?x) ; Q(?x)
R(?x)
True
True


**_Expected Output:_**

```
P(?x) ; Q(?x)
R(?x)
True
True
```

In [32]:
def equal_terms(t1, t2):
    if is_constant(t1) and is_constant(t2):
        return get_value(t1) == get_value(t2)
    if is_variable(t1) and is_variable(t2):
            return get_name(t1) == get_name(t2)
    if is_function_call(t1) and is_function(t2):
        if get_head(t1) != get_head(t2):
            return all([equal_terms(get_args(t1)[i], get_args(t2)[i]) for i in range(len(get_args(t1)))])
    return False

def is_equal_to(a1, a2):
    # We check atoms with the same predicate name and the same number of arguments
    if not (is_atom(a1) and is_atom(a2) and get_head(a1) == get_head(a2) and len(get_args(a1)) == len(get_args(a2))):
        return False
    return all([equal_terms(get_args(a1)[i], get_args(a2)[i]) for i in range(len(get_args(a1)))])

## Prove theorems by forward chaining

### Task 4

Implement the `apply_rule (rule, facts)` function which receives a rule and a set of facts and returns all the facts that can be determined by applying the rule to the given facts.

Use `unify`,` substitute`, but also `get_premises` and` get_conclusion` implemented earlier.

In [41]:
from copy import deepcopy
# from __future__ import print_function

def apply_rule(rule, facts):
    #TO DO
    
    resulting_facts = []
    premises = get_premises(rule)
    #print("Premises ", premises)
    conclusion = get_conclusion(rule)
    substitutions = []
    new_facts = []
    
    for premise in premises:
        #print("Premise ", premise)
        new_substitutions = []
        for fact in facts:
            res = unify(premise, fact)
            if res:
                check = True
                for key in res:
                    for subst in substitutions:
                        if key in subst.keys():
                            #print("In1")
                            check = False
                for key in res:
                    for subst in substitutions:
                        if key in subst.keys() and res[key]==subst[key]:
                            check = True
                if check:
                    new_substitutions.append(res)
        #print("New substitutions ", new_substitutions)
        for subst in substitutions:
            check = False
            for key in subst:
                #print(key)
                for new_subst in new_substitutions:
                    #print("new subst", new_subst)
                    if key in new_subst.keys():
                        #print("In")
                        check = True
            if check and subst not in new_substitutions:
                substitutions.remove(subst)
        for new_subst in new_substitutions:
            if new_subst not in substitutions:
                substitutions.append(new_subst)
        #print("Substitutions ", substitutions)
                
    for subst in substitutions:
        new_facts.append(substitute(conclusion, subst))
    #print(new_facts)
        
    for nf in new_facts:
        keep = True
        args = get_args(nf)
        for arg in args:
            if not is_constant(arg):
                keep = False
        if keep:
            resulting_facts.append(nf)
            
    return resulting_facts

# Test!
# Rule: P(x) => Q(x)
# Facts: P(1)
for f in apply_rule( 
        make_or(make_neg(make_atom("P", make_var("x"))), make_atom("Q", make_var("x"))), \
        [make_atom("P", make_const(1))]):
    print_formula(f) # should be Q(1)
print("=====")
# Rule: P(x) ^ Q(x) => R(x)
# Facts: P(1), P(2), P(3), Q(3), Q(2)
for f in apply_rule( 
        make_or(
            make_neg(make_atom("P", make_var("x"))),
            make_neg(make_atom("Q", make_var("x"))),
            make_atom("R", make_var("x"))),
        [make_atom("P", make_const(x)) for x in [1, 2, 3]] + \
        [make_atom("Q", make_const(x)) for x in [3, 2]]):
    print_formula(f) # should be R(2) and R(3)
print("=====")
# Rule: P(x) ^ Q(y) ^ R(x, y) => T(x, y)
# Facts: P(1), P(2), P(3), Q(3), Q(2), R(3, 2)
for f in apply_rule( 
        make_or(
            make_neg(make_atom("P", make_var("x"))),
            make_neg(make_atom("Q", make_var("y"))),
            make_neg(make_atom("R", make_var("x"), make_var("y"))),
            make_atom("T", make_var("x"), make_var("y"))),
        [make_atom("P", make_const(x)) for x in [1, 2, 3]] + \
        [make_atom("Q", make_const(x)) for x in [3, 2]] + \
        [make_atom("R", make_const(3), make_const(2))]):
    print_formula(f) # should be T(3, 2)

Q(1)
=====
R(2)
R(3)
=====
T(3, 2)


**_Expected Output:_**

```
Q(1)
=====
R(2)
R(3)
=====
T(3, 2)
```

In [42]:
def forward_chaining(kb, theorem, verbose = True):
    # We save the original database, we work with a copy
    local_kb = deepcopy(kb)
    # Two variables that describe the search status
    got_new_facts = True   # new facts were found in the last search
    is_proved = False      # the theorem has been proved
    # We check if the theorem is already proved
    for fact in filter(is_fact, local_kb):
        if unify(fact, theorem):
            if verbose: print("This already in KB: " + print_formula(fact, True))
            is_proved = True
            break
    while (not is_proved) and got_new_facts:
        got_new_facts = False
        for rule in filter(is_rule, local_kb):
            # For each rule:
            new_facts = apply_rule(rule, list(filter(is_fact, local_kb)))
            new_facts = list(filter(lambda fact: not any(list(filter(lambda orig: is_equal_to(fact, orig), local_kb))), new_facts))
            if new_facts:
                if verbose: print("Applied rule: " + print_formula(rule, True))
                got_new_facts = True
                for fact in new_facts:
                    #if verbose: print("New fact: " + print_formula(fact, True))
                    if unify(fact, theorem) != False:
                        is_proved = True
                        add_statement(local_kb, fact)
                        if verbose: print("Now in KB: " + print_formula(fact, True))
                        break
                    add_statement(local_kb, fact)
            if is_proved:
                break
    if verbose:
        if is_proved:
            print("The theorem is TRUE!")
        else:
            print("The theorem is FALSE!")
    return is_proved

In [43]:
def test_result(result, truth, idx):
    print("Test " + str(idx) + " OK!" if result == truth else "Test FAILED!")
    print("================== ")

test_result(forward_chaining(deepcopy(skb), make_atom("Sunny", make_var("x")), True), True, 0)
test_result(forward_chaining(deepcopy(skb), make_atom("Rain", make_var("x")), True), True, 1)
test_result(forward_chaining(deepcopy(skb), make_atom("Rain", make_const("Thursday")), True), True, 2)
test_result(forward_chaining(deepcopy(skb), make_atom("Sunny", make_const("Saturday")), True), True, 3)
test_result(forward_chaining(deepcopy(skb), make_atom("Activity",
                        make_const("Kevin"), make_var("Sport"), make_const("Saturday")), True), True, 4)

This already in KB: Sunny(Monday)
The theorem is TRUE!
Test 0 OK!
This already in KB: Rain(Friday)
The theorem is TRUE!
Test 1 OK!
The theorem is FALSE!
Test FAILED!
The theorem is FALSE!
Test FAILED!
The theorem is FALSE!
Test FAILED!


# Feedback

Hi!

I got Task 4 to work, but most of the tests are failed and I don't know why :/
Do you have any idea on what I should change?

See you in class! :)

Irina